In [2]:
import numpy as np
file = np.load(r"D:\Demonstrate\Dilithium\attack_show\coefficient_result.npy")
print(file.shape)
print(file[0])

(30, 500)
[193 195  -1  -1  -1 212 177  -1 150   6  50 149 108 227  53 137 251 209
 178  -1  25 153 195 145 140  82  -1   3 138 207 166   8  52  60 123  41
  73  -1   7  -1   3 123 242  32 218 101  82 201  -1   6  76 238 209  45
 135  -1 212 115  -1  -1 227  98  37  -1 137  25   3 109 155  -1  66  -1
  -1 185 227  52 230 170  -1 205 142 144 203  -1  16 227 102 144 122  -1
 170 252  -1 224  94   7 168  59  15 106  87  -1 183  -1  81  89 222  -1
  -1  34  39  78  54  -1 181 127  43  91 154 121  68 183 126 251 231 176
  53  -1 108   7  58 141  -1   5 249 107  -1  43  -1  -1  26 113 125  -1
 176 203  97 104 164 190   6  67 130  44  37  35  24  57 133 145  79  50
 224  -1 187 168 154  41  -1 143 155  58  -1 162 187 218  52  68 179 123
 235 117  -1  15 245 159 163  31  58  -1  -1 129 173  -1  -1 135 164 211
  14 230  61 239  -1  -1  -1 196 134  73  14 179  -1 123 140  84  50  81
 116 110  42 127  -1 118 182 240 185  -1  -1 123  64  13  -1  13  -1  17
 245 133  20  -1 158 213  -1  97  -1  -1 

In [1]:
## 根据预测的结果，输出0-1标志矩阵： 0表示预测结果有效；1表示预测的结果无效
def limit_fun(result):
    file_num = result.shape[0]     ## 多少个文件
    trace_num = result.shape[1]    ## 每个文件有多少条曲线
    limit_index = np.zeros([file_num*trace_num],dtype = int)
    for i in range(file_num):
        for j in range(trace_num):
            if result[i,j] == -1:
                limit_index[i*trace_num+j] = 1
    return limit_index

In [2]:
import numpy as np
from tqdm import trange
import numba as nb
import time
import sys
sys.path.append(r"D:\Demonstrate\Dilithium\attack_show\dilithium-py-main")   ## 注意这里
from dilithium import Dilithium,DEFAULT_PARAMETERS

## 解包函数
def Z_unpack(blocknum,label_file_url):   # blocknum是块数
    D=Dilithium(DEFAULT_PARAMETERS["dilithium2"])
    data=np.load(label_file_url+"\meta_data_part{0}.npz".format(blocknum),allow_pickle=True)
    sm=data['sm']
    Z_array=np.zeros([len(sm),4,256],dtype=int)
    # 每一条曲线的z的多项式系数，一个z是4*256的
    C_array=np.zeros([len(sm),256],dtype=int)
    # 每一条曲线的c的多项式系数，一个z是1*256的

    for j in range(len(sm)):#对每一条曲线进行遍历
        z=bytes.fromhex(sm[j][:4840])
        C,Z_poly,h=D._unpack_sig(sig_bytes=z)
        C_tall=D._sample_in_ball(C)
        C_array[j]=np.array(C_tall.coeffs,dtype=int)
        for p in range(4):
            Z_array[j][p]=np.array(Z_poly[p][0].coeffs,dtype=int)
    return C_array,Z_array
##########   带模的多项式乘法    ##########################
def poly_multi(X,m):
    n = X.shape[0]
    D1 = np.zeros((n,n),dtype = int)  
    D2 = np.zeros((n,n),dtype = int)
    
    for j in range(m+1):    ## 矩阵运算法    coeff_front
        D1[m][j] = X[m-j]

    X = X[::-1]
    for j in range(m+1,n):   ## 矩阵运算法    生成coeff_behind
        D2[m][j] = X[j-m-1]

    s1_coeff = (D1-D2)[m]
    return s1_coeff 

## 获取Z值函数： 得到每条trace上y=0所对应的z值
def get_z_ij(Z,index,predict,file_num):    
    z_ij = np.zeros((500,2),dtype = int)
    for i in range(500):
        z_ij[i,0] = Z[i,index[i],predict[i]]
        z_ij[i,1] = index[i]
    #np.save(r"D:\Introduction\竞赛\data\result\z_ij_coeff_part{0}.npy".format(file_num),z_ij)
    return z_ij   ##(500,2)

## 获得一个npz文件(500条曲线)的S1的系数
def combine_coeff(C,predict,file_num):
    cs1_coeff = np.zeros((500,256),dtype=int)
    
    for i in trange(500):
        cs1_coeff[i] = poly_multi(C[i],predict[i])
    return cs1_coeff    ## (500,256)

def get_coeff_of_CS_z(file_number,label_file_url,predict_file):
    final_data = np.zeros([500*file_number,258],dtype = int)     ## 最终get_coeff_of_CS_z输出文件，把z_ij和CS1合并
    for file_num in range(file_number):
        
        label = np.load(label_file_url+"\meta_data_part{0}.npz".format(file_num),allow_pickle=True)
        predict_file = predict_file   ## (10,500)  这里是多个预测文件合并为一个；后期可能调整
        C,Z = Z_unpack(file_num,label_file_url)     # (500, 256)  # (500, 4, 256)   
        
        index = label['index']   # (500,)
        predict = predict_file[file_num]    ## 后期可能调整

        final_data[500*file_num:500*file_num+500,256:] = get_z_ij(Z,index,predict,file_num)   ## step1. 获得Z_ij的值  (500,2)
        final_data[500*file_num:500*file_num+500,0:256] = combine_coeff(C,predict,file_num)    ## step2. 获得CS1的值  (500,256)
    return final_data     # (500*file_number, 258)

## 把CS1系数和Z_ij的组合矩阵按照index划分为4组
def split_cs1_coeff(final_data,index_error):
    csz_0 = np.zeros((1,257),dtype = int)   ##  用来存放第一个多项式的 C*S1的系数 和 z值（前256是系数，最后一个是z值）
    csz_1 = np.zeros((1,257),dtype = int)   ##  用来存放第二个...
    csz_2 = np.zeros((1,257),dtype = int)
    csz_3 = np.zeros((1,257),dtype = int)
    
    for i in range(final_data.shape[0]):
        if index_error[i] == 0:             ## 检验位，排除预测为-1的
            if final_data[i,-1] == 0:
                csz_0 = np.vstack((csz_0,final_data[i,:257]))
            elif final_data[i,-1] == 1:
                csz_1 = np.vstack((csz_1,final_data[i,:257]))
            elif final_data[i,-1] == 2:
                csz_2 = np.vstack((csz_2,final_data[i,:257]))
            else:
                csz_3 = np.vstack((csz_3,final_data[i,:257]))
    print("每组trace数量：",csz_0.shape,"  ",csz_1.shape,"  ",csz_2.shape,"   ",csz_3.shape)  ##检查每个分类多少条
    return csz_0[1:],csz_1[1:],csz_2[1:],csz_3[1:]   # (500*file_number, 257)

### 最小二乘法
def leastSquares(cz_i,use_trace_num):   
    c = cz_i[:use_trace_num,:256]
    z = cz_i[:use_trace_num,-1]
    meddle = np.dot(c.T,c)  # c^T  * c
    meddle2 = np.linalg.inv(meddle) # (c^T  * c)^-1
    meddle3 = np.dot(meddle2,c.T)  # (C^T * C)^-1  *  C^T
    
    return np.dot(meddle3,z)   # (C^T * C)^-1  *  C^T  * Z

## 检验准确率
def accuracy(predict_S1,real_S1_url,group_num,which):
    real_S1 = np.load(real_S1_url)[group_num][which]
    n = predict_S1.shape[0]
    count = 0
    predict_S1 = np.round(predict_S1)
    for i in range(n):
        if predict_S1[i] - real_S1[i] == 0:
            count += 1
    accuracy = count/n
    return(accuracy)

def main(file_number,label_file_url,predict_file,use_trace_num,index_error):
    final_data = get_coeff_of_CS_z(file_number,label_file_url,predict_file) 
    csz_0,csz_1,csz_2,csz_3 = split_cs1_coeff(final_data,index_error)
    predict_S1_0 = leastSquares(csz_0,use_trace_num)
    predict_S1_1 = leastSquares(csz_1,use_trace_num)
    predict_S1_2 = leastSquares(csz_2,use_trace_num)
    predict_S1_3 = leastSquares(csz_3,use_trace_num)
    return predict_S1_0,predict_S1_1,predict_S1_2,predict_S1_3
##################################################################################################################################
######################################### API ####################################################################################
if __name__ == '__main__':
    group_number = 0       ###  攻击第几组
    file_number =  3     ## 使用多少个npy文件
    use_trace_num =   500                           ## 使用多少数量trace
             
    label_file_url = r"D:\Demonstrate\Dilithium\attack_show\meta_data"                                             
    predict_file =  np.load(r"D:\Demonstrate\Dilithium\attack_show\coefficient_result.npy")#result
    index_error  =  limit_fun(predict_file)#np.load(r"index_error.npy")#index_error
######################################    output   #############################################
    predict_S1_0,predict_S1_1,predict_S1_2,predict_S1_3 = main(file_number,label_file_url,predict_file,use_trace_num,index_error) 
##################  预测结果写入txt文件  #######################################################
#     predict_S1 = np.hstack((predict_S1_0,predict_S1_1,predict_S1_2,predict_S1_3))
#     print("shape:",predict_S1.shape)
#     np.savetxt('predict_s1_group{}.txt'.format(group_number), predict_S1, fmt='%d')
##################   检验准确率   ##############################################################
    real_S1_url = r"D:\Demonstrate\Dilithium\attack_show\s1.npy"
    accuracy(predict_S1_0,real_S1_url,0,0)
    accuracy(predict_S1_1,real_S1_url,0,1)
    accuracy(predict_S1_2,real_S1_url,0,2)
    accuracy(predict_S1_3,real_S1_url,0,3)
    
    print("***第{}组S1的第0个系数***".format(group_number))
    print("使用曲线{}条".format(use_trace_num),"        预测准确率：",accuracy(predict_S1_0,real_S1_url,group_number,0))
    print("***第{}组S1的第1个系数***".format(group_number))
    print("使用曲线{}条".format(use_trace_num),"        预测准确率：",accuracy(predict_S1_1,real_S1_url,group_number,1))
    print("***第{}组S1的第2个系数***".format(group_number))
    print("使用曲线{}条".format(use_trace_num),"        预测准确率：",accuracy(predict_S1_2,real_S1_url,group_number,2))
    print("***第{}组S1的第3个系数***".format(group_number))
    print("使用曲线{}条".format(use_trace_num),"        预测准确率：",accuracy(predict_S1_3,real_S1_url,group_number,3))

100%|██████████| 500/500 [00:00<00:00, 10333.70it/s]

每组trace数量： (316, 257)    (298, 257)    (295, 257)     (287, 257)
***第0组S1的第0个系数***
使用曲线500条         预测准确率： 1.0
***第0组S1的第1个系数***
使用曲线500条         预测准确率： 1.0
***第0组S1的第2个系数***
使用曲线500条         预测准确率： 1.0
***第0组S1的第3个系数***
使用曲线500条         预测准确率： 1.0


In [3]:
real = np.load(r"D:\Demonstrate\Dilithium\attack_show\s1.npy")
print(real.shape)
print(real[0,0])
print(real[1,0])

(10, 4, 256)
[-1 -1 -1  1 -1  2  0 -1  1 -2  1 -1  1  1  0  1 -1 -2 -2 -1  2 -1 -2  2
 -2 -1 -2  1 -1 -2  2  1  2 -1  1  0  1  1  0  0 -2 -1 -2  2  2  2 -1  0
 -1 -2 -2  0  1  0  0  0  2 -1  1 -2  0  1  2 -1  1  0  0  2  0 -1 -1  2
 -1  0 -2  2  1 -1 -2 -1  0  0  1  2 -2  2  2  2  0  1  2 -1  0  1  2  1
  1  1  1  0  1 -2  0  1  2  1  2  2  0  2 -1 -1 -1  0  2  1 -2 -2 -2  1
  0 -2  0  2  2  2 -1  0 -2 -1 -1 -1  1  0 -1 -2 -2  1  1  2 -2 -2 -2 -2
  2  2  2 -2  0  1  0 -2  2  2  0  0 -2 -1  0  1  0 -2 -2 -1 -2 -2 -2 -2
  2  0  1  2  1 -1  0 -1  2 -2  1 -2 -1  1 -1  0 -1  0  0  1  2  0  0  2
  2  2 -2 -1 -2 -1  1  2  2  1  1  2 -1 -2 -1  1  0  2  1  0  0 -2  1  1
 -2 -1  2  2 -2 -1  0  1  1 -2 -2 -2  2  2  0  2 -2 -2  2 -2 -2 -2  0  1
 -1  2  2  1  0 -2  1 -2  0 -1 -1  0  2 -1  1 -2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 